# Calling C/Fortran Code from Julia

Guides:
- https://docs.julialang.org/en/v1/manual/calling-c-and-fortran-code/


In [1]:
print("Julia version is:")
VERSION

Julia version is:

v"1.4.1"

# C

Code contents:

In [2]:
;cat example.c

#include <stdio.h>

float funcAdd(float input) {
    return input + 1.5;
}


int main() {
    printf("Hello World!");
    printf("%g", funcAdd(3.0));
    return 0;
}


Compile

In [3]:
;gcc -shared -fPIC example.c -o example_c.so

See the contents of the shared library:

In [4]:
;nm -D example_c.so

                 w __cxa_finalize
0000000000001139 T funcAdd
                 w __gmon_start__
                 w _ITM_deregisterTMCloneTable
                 w _ITM_registerTMCloneTable
0000000000001159 T main
                 U printf


Call it from Julia:

In [5]:
println("Calling with a variable:")
a = 1.0
println(ccall((:funcAdd, "./example_c.so"), Float32, (Float32,), a,))
println("Calling with a constant:")
println(ccall((:funcAdd, "./example_c.so"), Float32, (Float32,), 1.0,))
println("Calling with the wrong type (Float64 instead of the correct Float32).")
println(ccall((:funcAdd, "./example_c.so"), Float64, (Float64,), 1.0,))

Calling with a variable:
2.5
Calling with a constant:
2.5
Calling with the wrong type (Float64 instead of the correct Float32).
5.28426686e-315


# Fortran

Code contents:

In [6]:
;cat example.f90

program program_name
    implicit none
! This is a comment
    real :: out
    real :: funcAdd

    out = funcAdd(1.0)
    print *,  out
end program program_name

real function funcAdd(param)
    real :: param
    funcAdd = param + 1.5
end

Compile to shared libraries.

In [7]:
;gfortran -shared -fPIC example.f90 -o example_f.so

See the contents of the shared library:

In [8]:
;nm -D example_f.so

                 w __cxa_finalize
0000000000001159 T funcadd_
                 U _gfortran_set_args
                 U _gfortran_set_options
                 U _gfortran_st_write
                 U _gfortran_st_write_done
                 U _gfortran_transfer_real_write
                 w __gmon_start__
                 w _ITM_deregisterTMCloneTable
                 w _ITM_registerTMCloneTable
0000000000001207 T main


Useful reference about name mangling and `Ref` is available in the official `ccall` document
([here](https://docs.julialang.org/en/v1/manual/calling-c-and-fortran-code/), [Fortran example section](https://docs.julialang.org/en/v1/manual/calling-c-and-fortran-code/#Fortran-Wrapper-Example-1)).

Run:

In [9]:
println("Calling with a variable (without Ref):")
a = 1.0
println(ccall((:funcadd_, "./example_f.so"), Float32, (Float32,), a,))
println("Calling with a variable (with Ref):")
println(ccall((:funcadd_, "./example_f.so"), Float32, (Ref{Float32},), a,))
println("Calling with the wrong type (Float64 instead of the correct Float32).")
println(ccall((:funcadd_, "./example_f.so"), Float64, (Ref{Float64},), a,))

Calling with a variable (without Ref):
1.5
Calling with a variable (with Ref):
2.5
Calling with the wrong type (Float64 instead of the correct Float32).
5.28426686e-315


In [10]:
println("Call it with a constant:")
println(ccall((:funcadd_, "./example_f.so"), Float32, (Ref{Float32},), 1.0,))

Call it with a constant:
2.5


But this will cause Segmentation fault, and if run it will cause the Jupyter notebook's kernel to die:

```julia
println(ccall((:funcadd_, "./example_f.so"), Float32, (Float32,), 1.0,))
```